## 5.2.4

In [5]:
function initialize_spins(Lx,Ly)
    return rand([-1,1],Lx,Ly)
end

initialize_spins (generic function with 1 method)

In [6]:
function measure_Mz(Ck)
    return sum(Ck)
end

measure_Mz (generic function with 1 method)

In [7]:
function measure_energy(Ck,J,h,Lx,Ly)
    energy = 0
    for iy=1:Ly
        for ix=1:Lx
            Si = calc_Si(ix,iy,Lx,Ly,Ck)
            σi = Ck[ix,iy]
            energy += -(J/2)*σi*Si - h*σi
        end
    end
    return energy
end

measure_energy (generic function with 1 method)

In [8]:
function calc_Si(ix,iy,Lx,Ly,Ck)
    jx = ix + 1
    if jx > Lx
        jx -= Lx
    end
    jy=iy
    Si = Ck[jx,jy]
    
    jx = ix - 1
    if jx < 1
        jx += Lx
    end
    jy = iy
    Si += Ck[jx,jy]
    
    jy = iy + 1
    if jy > Ly
        jy -= Ly
    end
    jx = ix
    Si += Ck[jx,jy]
    
    jy = iy-1
    if jy < 1
        jy += Ly
    end
    jx = ix
    Si += Ck[jx,jy]
    return Si
end

calc_Si (generic function with 1 method)

In [9]:
function calc_ΔE(Ck,ix,iy,J,h,Lx,Ly)
    Si = calc_Si(ix,iy,Lx,Ly,Ck)
    return 2J*Ck[ix,iy]*Si + 2h*Ck[ix,iy]
end

calc_ΔE (generic function with 1 method)

In [10]:
function metropolis(σi,ΔE,T)
    is_accepted = ifelse(rand() <= exp(-ΔE/T),true,false)
    σ_new = ifelse(is_accepted,-σi,σi)
    return σ_new,is_accepted
end

metropolis (generic function with 1 method)

In [11]:
function heatbath(σi,ΔE,T)
    α = ΔE*σi
    σ_new = ifelse(rand() <= 1/(1+exp(-α/T)),+1,-1)
    is_accepted = ifelse(σ_new == σi,false,true)
    return σ_new,is_accepted
end


heatbath (generic function with 1 method)

In [12]:
function local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    ΔE = calc_ΔE(Ck,ix,iy,J,h,Lx,Ly)
    σi = Ck[ix,iy]
    return metropolis(σi,ΔE,T)
end
function local_heatbath_update(Ck,ix,iy,T,J,h,Lx,Ly)
    ΔE = calc_ΔE(Ck,ix,iy,J,h,Lx,Ly)
    σi = Ck[ix,iy]
    return heatbath(σi,ΔE,T)
end

local_heatbath_update (generic function with 1 method)

In [13]:
using Random
using Plots
function montecarlo(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    Random.seed!(123)
    num_total = num_thermal+num_MC
    accept_count = 0
    absmz_meanvalue = 0
    measure_count = 0
    mz_data = []
    update(Ck,ix,iy) = local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    
    Ck = initialize_spins(Lx,Ly)
    
    for trj = 1:num_total
        for isweep = 1:Lx*Ly
            ix = rand(1:Lx)
            iy = rand(1:Ly)
            Ck[ix,iy],is_accepted = update(Ck,ix,iy)
            
            accept_count += ifelse(is_accepted,1,0)
        end
        
        if trj > num_thermal
            if trj % measure_interval == 0
                measure_count += 1
                mz = measure_Mz(Ck)/(Lx*Ly)
                absmz_meanvalue += abs(mz)
                push!(mz_data,mz)
            end
        end 
    end
    return mz_data,accept_count/(num_total*Lx*Ly),absmz_meanvalue/measure_count
                
end

montecarlo (generic function with 1 method)

In [14]:
function test()
    Lx = 100
    Ly = 100
    J = 1
    h = 0
    num_thermal = 200
    num_MC = 10000-num_thermal
    measure_interval = 10
    T = 1
    @time mz_data,acceptance_ratio,absmz = montecarlo(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    println("average acceptance ratio ",acceptance_ratio)
    histogram(mz_data,bin=-1:0.01:1)
    savefig("mz_data_$T.png")
    return
end
test()

  4.054132 seconds (998 allocations: 125.898 KiB)
average acceptance ratio 0.00181302


In [15]:
function montecarlo_fast(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    Random.seed!(123)
    num_total = num_thermal+num_MC
    accept_count = 0
    absmz_meanvalue = 0
    measure_count = 0
    mz_data = []
    update(Ck,ix,iy) = local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    
    Ck = initialize_spins(Lx,Ly)
    
    for trj = 1:num_total
        for ix = 1:Lx
            for iy=1:Ly
                Ck[ix,iy],is_accepted = update(Ck,ix,iy)
            
                accept_count += ifelse(is_accepted,1,0)
            end
        end
        
        if trj > num_thermal
            if trj % measure_interval == 0
                measure_count += 1
                mz = measure_Mz(Ck)/(Lx*Ly)
                absmz_meanvalue += abs(mz)
                push!(mz_data,mz)
            end
        end 
    end
    return mz_data,accept_count/(num_total*Lx*Ly),absmz_meanvalue/measure_count
                
end

montecarlo_fast (generic function with 2 methods)

In [12]:
function test_tdep()
    Lx = 100
    Ly = 100
    J = 1
    h = 0
    num_thermal = 5000
    num_MC = 50000-num_thermal
    measure_interval = 10
    mz_Tdep = []
    nT = 20
    Ts = range(0.5,4.0,length = nT)
    for T in Ts
        @time mz_data,acceptance_ratio,absmz = montecarlo_fast(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
        push!(mz_Tdep,absmz)
        println("$T $absmz")
        histogram(mz_data,bin=-1:0.01:1)
        savefig("mz_data_$(T).png")
    end
    plot(Ts,mz_Tdep)
    savefig("mz_tdep.png")
    return
end

test_tdep (generic function with 1 method)

In [13]:
test_tdep()

 18.427934 seconds (4.52 k allocations: 277.336 KiB)
0.5 0.05199453333333343
 18.698643 seconds (4.52 k allocations: 277.336 KiB)
0.6842105263157895 0.23100075555555571
 18.489597 seconds (4.52 k allocations: 277.336 KiB)
0.868421052631579 0.9997946222222384
 18.601177 seconds (4.52 k allocations: 277.336 KiB)
1.0526315789473684 0.9988989777777715
 18.372596 seconds (4.52 k allocations: 277.336 KiB)
1.236842105263158 0.9962966666666621
 18.643263 seconds (4.52 k allocations: 277.336 KiB)
1.4210526315789473 0.9755179555555478
 18.486626 seconds (4.52 k allocations: 277.336 KiB)
1.605263157894737 0.9791694222222268
1031.705697 seconds (4.52 k allocations: 277.336 KiB)
1.7894736842105263 0.9584731555555526
 18.315030 seconds (4.52 k allocations: 277.336 KiB)
1.9736842105263157 0.9195799555555548
 18.242392 seconds (4.52 k allocations: 277.336 KiB)
2.1578947368421053 0.8301916000000004
 18.593858 seconds (4.52 k allocations: 277.336 KiB)
2.3421052631578947 0.16207084444444436
 18.476981 se

In [16]:
function montecarlo_fast(filename,num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    ENV["GKSwstype"] = "nul"
    Random.seed!(123)
    num_total = num_thermal+num_MC
    accept_count = 0
    absmz_meanvalue = 0
    measure_count = 0
    mz_data = []
    update(Ck,ix,iy) = local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    
    Ck = initialize_spins(Lx,Ly)
    
    ising = @animate for trj = 1:num_total
        for ix = 1:Lx
            for iy=1:Ly
                Ck[ix,iy],is_accepted = update(Ck,ix,iy)
            
                accept_count += ifelse(is_accepted,1,0)
            end
        end
        
        if trj > num_thermal
            if trj % measure_interval == 0
                measure_count += 1
                mz = measure_Mz(Ck)/(Lx*Ly)
                absmz_meanvalue += abs(mz)
                push!(mz_data,mz)
            end
        end 
        heatmap(1:Lx,1:Ly,Ck,aspect_ratio=:equal)
    end every 100
    gif(ising, "./"*filename, fps = 15)  
    return mz_data,accept_count/(num_total*Lx*Ly),absmz_meanvalue/measure_count
                
end

montecarlo_fast (generic function with 2 methods)

In [17]:
function test_anime()
    Lx = 100
    Ly = 100
    J = 1
    h = 0
    num_thermal = 5000
    num_MC =20000-num_thermal
    measure_interval = 10
    T = 0.5
    @time mz_data,acceptance_ratio,absmz = montecarlo_fast("ising_T$T.gif",num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)

    println("average acceptance ratio ",acceptance_ratio)
    histogram(mz_data,bin=-1:0.01:1)
    savefig("mz_data_$(T).png")
    return
end
test_anime()

 25.287235 seconds (63.65 M allocations: 5.784 GiB, 2.56% gc time, 7.50% compilation time)
average acceptance ratio 0.01565267


┌ Info: Saved animation to 
│   fn = /Users/yuki/git/YukiNagai/docs/src_ja/books/ising_T0.5.gif
└ @ Plots /Users/yuki/.julia/packages/Plots/UeTBV/src/animation.jl:114


In [18]:
function montecarlo_fast(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
    Random.seed!(123)
    num_total = num_thermal+num_MC
    accept_count = 0
    absmz_meanvalue = 0
    measure_count = 0
    mz_data = []
    update(Ck,ix,iy) = local_metropolis_update(Ck,ix,iy,T,J,h,Lx,Ly)
    E2_meanvalue = 0.0 
    E_meanvalue = 0.0 
    
    Ck = initialize_spins(Lx,Ly)
    
    for trj = 1:num_total
        if trj > num_thermal && rand() < 0.01 
            @. Ck *= -1
            continue
        end
        
        for ix = 1:Lx
            for iy=1:Ly
                Ck[ix,iy],is_accepted = update(Ck,ix,iy)
            
                accept_count += ifelse(is_accepted,1,0)
            end
        end
        
        if trj > num_thermal
            if trj % measure_interval == 0
                measure_count += 1
                mz = measure_Mz(Ck)/(Lx*Ly)
                absmz_meanvalue += abs(mz)
                push!(mz_data,mz)
                
                E = measure_energy(Ck,J,h,Lx,Ly)
                E2_meanvalue += E^2
                E_meanvalue += E
                
            end
        end 
    end
    Cv = (E2_meanvalue/measure_count - (E_meanvalue/measure_count)^2)/T^2

    return mz_data,accept_count/(num_total*Lx*Ly),absmz_meanvalue/measure_count,Cv
                
end

montecarlo_fast (generic function with 2 methods)

In [19]:

function test_tdep()
    Lx = 96
    Ly = 96
    J = 1
    h = 0
    num_thermal = 20000
    num_MC =100000-num_thermal
    measure_interval = 10
    mz_Tdep = []
    Cv_Tdep = []

    nT = 20
    Ts = range(0.5,4.0,length= nT)
    for T in Ts
        @time mz_data,acceptance_ratio,absmz,Cv = montecarlo_fast(num_thermal,num_MC,measure_interval,T,J,h,Lx,Ly)
        push!(mz_Tdep,absmz)
        push!(Cv_Tdep,Cv)
        println("$T $absmz, $Cv")
        histogram(mz_data,bin=-1:0.01:1)
        savefig("mz_data_L$(Lx)_T$(T).png")

        plot(mz_data)
        savefig("mz_trjdep_L$(Lx)_T$(T).png")
    end
    plot(Ts,mz_Tdep)
    savefig("mz_tdep_L$(Lx).png")

    plot(Ts,Cv_Tdep)
    savefig("Cv_tdep_L$(Lx).png")
    return
end

test_tdep (generic function with 1 method)

In [20]:
test_tdep()

histgoram
 33.000136 seconds (7.95 k allocations: 324.789 KiB)
0.5 0.9999996715643, 0.3868520259857178
 32.711374 seconds (7.95 k allocations: 324.789 KiB)
0.6842105263157895 0.999983085561452, 10.428095578441958
 33.014732 seconds (7.95 k allocations: 324.789 KiB)
0.868421052631579 0.9997949192749589, 79.50156693244001
 32.766487 seconds (7.95 k allocations: 324.789 KiB)
1.0526315789473684 0.9988974139848755, 294.54911037564284
 32.712517 seconds (7.95 k allocations: 324.789 KiB)
1.236842105263158 0.9963071510961877, 718.7639276587406
 32.816952 seconds (7.95 k allocations: 324.789 KiB)
1.4210526315789473 0.9905089198757029, 1419.4747793447987
 33.119237 seconds (7.95 k allocations: 324.789 KiB)
1.605263157894737 0.979192284651277, 2529.8853416078928
 49.414598 seconds (7.95 k allocations: 324.789 KiB)
1.7894736842105263 0.9584856453892898, 3984.776147137515
 32.682030 seconds (7.95 k allocations: 324.789 KiB)
1.9736842105263157 0.9194003191081269, 6396.2623672445725
 32.789297 second